# Experiment 01: Airline dataset

In this experiment we use [the airline dataset](http://kt.ijs.si/elena_ikonomovska/data.html) to predict arrival delay. The dataset consists of a large amount of records, containing flight arrival and departure details for all the commercial flights within the USA, from October 1987 to April 2008. Its size is around 116 million records and 5.76 GB of memory.



In [1]:
import os,sys
import numpy as np
import pandas as pd
from lightgbm.sklearn import LGBMRegressor, LGBMClassifier
from xgboost import XGBRegressor
from sklearn.metrics import (confusion_matrix, accuracy_score, roc_auc_score, f1_score, log_loss, precision_score,
                             recall_score)
from libs.loaders import load_airline
from libs.conversion import convert_cols_categorical_to_numeric, convert_related_cols_categorical_to_numeric
from libs.timer import Timer
import pkg_resources
import json
import matplotlib.pylab as plt

print("System version: {}".format(sys.version))
print("XGBoost version: {}".format(pkg_resources.get_distribution('xgboost').version))
print("LightGBM version: {}".format(pkg_resources.get_distribution('lightgbm').version))

%matplotlib inline

System version: 3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
XGBoost version: 0.6
LightGBM version: 0.2


# 1) XGBoost vs LightGBM benchmark
In the next section we compare both libraries speed, accuracy and other metrics for the dataset of airline arrival delay. 

### Data loading and management

In [2]:
%%time
df_plane = load_airline()
print(df_plane.shape)

MOUNT_POINT not found in environment. Defaulting to /fileshare
(115069017, 14)
CPU times: user 1min 12s, sys: 47 s, total: 1min 59s
Wall time: 4min 8s


In [3]:
df_plane.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,AA,190,247,SFO,ORD,1846,0,27
1,1987,10,1,4,5,114,EA,57,74,LAX,SFO,337,0,5
2,1987,10,1,4,5,35,HP,351,167,ICT,LAS,987,0,17
3,1987,10,1,4,5,40,DL,251,35,MCO,PBI,142,0,-2
4,1987,10,1,4,8,517,UA,500,208,LAS,ORD,1515,0,17


The first step is to convert the categorical features to numeric features.

In [4]:
%%time
df_plane_numeric = convert_related_cols_categorical_to_numeric(df_plane, col_list=['Origin','Dest'])
del df_plane

CPU times: user 1min 35s, sys: 12.2 s, total: 1min 48s
Wall time: 1min 50s


In [5]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,AA,190,247,0,33,1846,0,27
1,1987,10,1,4,5,114,EA,57,74,1,0,337,0,5
2,1987,10,1,4,5,35,HP,351,167,2,4,987,0,17
3,1987,10,1,4,5,40,DL,251,35,3,41,142,0,-2
4,1987,10,1,4,8,517,UA,500,208,4,33,1515,0,17


In [6]:
%%time
df_plane_numeric = convert_cols_categorical_to_numeric(df_plane_numeric, col_list='UniqueCarrier')


CPU times: user 50.9 s, sys: 38.5 s, total: 1min 29s
Wall time: 1min 31s


In [7]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,0,190,247,0,33,1846,0,27
1,1987,10,1,4,5,114,1,57,74,1,0,337,0,5
2,1987,10,1,4,5,35,2,351,167,2,4,987,0,17
3,1987,10,1,4,5,40,3,251,35,3,41,142,0,-2
4,1987,10,1,4,8,517,4,500,208,4,33,1515,0,17


To simplify the pipeline, we are going to set a classification problem where the goal is to classify wheather a flight has arrived delayed or not. For that we need to binarize the variable `ArrDelay`.

If you want to extend this experiment, you can set a regression problem and try to identify the number of minutes of delay a fight has. Both XGBoost and LightGBM have regression classes.

In [8]:
%%time
filter_func = lambda x: 1 if x > 0 else 0
df_plane_numeric['ArrDelayBinary'] = df_plane_numeric['ArrDelay'].map(filter_func)

CPU times: user 34.7 s, sys: 6.78 s, total: 41.5 s
Wall time: 42.3 s


In [9]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay,ArrDelayBinary
0,1987,10,1,4,1,556,0,190,247,0,33,1846,0,27,1
1,1987,10,1,4,5,114,1,57,74,1,0,337,0,5,1
2,1987,10,1,4,5,35,2,351,167,2,4,987,0,17,1
3,1987,10,1,4,5,40,3,251,35,3,41,142,0,-2,0
4,1987,10,1,4,8,517,4,500,208,4,33,1515,0,17,1


Once the features are prepared, let's split the dataset into train, validation and test set.

In [10]:
def split_train_val_test_df(df, val_size=0.2, test_size=0.2):
    train, validate, test = np.split(df.sample(frac=1), 
                                     [int((1-val_size-test_size)*len(df)), int((1-test_size)*len(df))])
    return train, validate, test

In [11]:
%%time
train, validate, test = split_train_val_test_df(df_plane_numeric)
print(train.shape)
print(validate.shape)
print(test.shape)

(69041410, 15)
(23013803, 15)
(23013804, 15)
CPU times: user 41.7 s, sys: 23.5 s, total: 1min 5s
Wall time: 1min 6s


In [12]:
def generate_feables(df):
    X = df[df.columns.difference(['ArrDelay', 'ArrDelayBinary'])]
    y = df['ArrDelayBinary']
    return X,y

In [13]:
%%time
X_train, y_train = generate_feables(train)
X_val, y_val = generate_feables(validate)
X_test, y_test = generate_feables(test)


CPU times: user 1.79 s, sys: 2.59 s, total: 4.38 s
Wall time: 4.47 s


### Training 
Now we are going to create two pipelines, one of XGBoost and one for LightGBM. The technology behind both libraries is different, so it is difficult to compare them in the exact same model setting. XGBoost grows the trees depth-wise and controls model complexity with `max_depth`. Instead, LightGBM uses a leaf-wise algorithm and controls the model complexity by `num_leaves`. As a tradeoff, we use XGBoost with `max_depth=8`, which will have max number leaves of 255, and compare it with LightGBM with `num_leaves=255`. 

In [14]:
results_dict = dict()

Let's start with the XGBoost classifier.

In [15]:
xgb_clf_pipeline = XGBRegressor(max_depth=8,
                                n_estimators=50,
                                min_child_weight=30,
                                learning_rate=0.1,
                                subsample=0.80,
                                colsample_bytree=0.80,
                                seed=77)

In [16]:
with Timer() as t:
    xgb_clf_pipeline.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=True, eval_metric='logloss')

[0]	validation_0-logloss:0.679804
[1]	validation_0-logloss:0.67054
[2]	validation_0-logloss:0.662075
[3]	validation_0-logloss:0.658527
[4]	validation_0-logloss:0.65019
[5]	validation_0-logloss:0.647835
[6]	validation_0-logloss:0.641422
[7]	validation_0-logloss:0.63599
[8]	validation_0-logloss:0.634052
[9]	validation_0-logloss:0.632017
[10]	validation_0-logloss:0.630585
[11]	validation_0-logloss:0.62913
[12]	validation_0-logloss:0.624305
[13]	validation_0-logloss:0.619909
[14]	validation_0-logloss:0.61576
[15]	validation_0-logloss:0.612229
[16]	validation_0-logloss:0.611062
[17]	validation_0-logloss:0.608002
[18]	validation_0-logloss:0.605401
[19]	validation_0-logloss:0.603999
[20]	validation_0-logloss:0.601453
[21]	validation_0-logloss:0.599196
[22]	validation_0-logloss:0.597714
[23]	validation_0-logloss:0.595868
[24]	validation_0-logloss:0.594864
[25]	validation_0-logloss:0.593114
[26]	validation_0-logloss:0.592256
[27]	validation_0-logloss:0.589761
[28]	validation_0-logloss:0.587943


In [17]:
results_dict['xgb']={
    'train_time': t.interval
}

Training LightGBM classifier

In [18]:
lgbm_clf_pipeline = LGBMRegressor(num_leaves=255,
                                 n_estimators=50,
                                 min_child_weight=30,
                                 learning_rate=0.1,
                                 subsample=0.80,
                                 colsample_bytree=0.80,
                                 seed=77,
                                 silent=False)

In [21]:
def loglikelood(y_true, y_pred):
    eval_result = log_loss(y_true, y_pred)
    eval_name = 'log_loss'
    is_bigger_better = False
    return eval_name, eval_result, is_bigger_better

In [22]:
with Timer() as t:
    lgbm_clf_pipeline.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=True, eval_metric=loglikelood)

[1]	valid_0's log_loss: 0.687301
[2]	valid_0's log_loss: 0.671971
[3]	valid_0's log_loss: 0.657799
[4]	valid_0's log_loss: 0.646254
[5]	valid_0's log_loss: 0.63618
[6]	valid_0's log_loss: 0.627507
[7]	valid_0's log_loss: 0.619924
[8]	valid_0's log_loss: 0.617961
[9]	valid_0's log_loss: 0.611274
[10]	valid_0's log_loss: 0.604965
[11]	valid_0's log_loss: 0.599788
[12]	valid_0's log_loss: 0.597928
[13]	valid_0's log_loss: 0.596354
[14]	valid_0's log_loss: 0.591256
[15]	valid_0's log_loss: 0.590064
[16]	valid_0's log_loss: 0.588472
[17]	valid_0's log_loss: 0.583923
[18]	valid_0's log_loss: 0.582721
[19]	valid_0's log_loss: 0.581553
[20]	valid_0's log_loss: 0.580321
[21]	valid_0's log_loss: 0.579135
[22]	valid_0's log_loss: 0.577977
[23]	valid_0's log_loss: 0.574284
[24]	valid_0's log_loss: 0.570868
[25]	valid_0's log_loss: 0.567731
[26]	valid_0's log_loss: 0.565246
[27]	valid_0's log_loss: 0.564268
[28]	valid_0's log_loss: 0.562781
[29]	valid_0's log_loss: 0.560805
[30]	valid_0's log_loss:

In [23]:
results_dict['lgbm']={
    'train_time': t.interval
}

As it can be seen in the results, given the specific versions and parameters of both XGBoost and LightGBM and in this specific dataset, LightGBM is faster. 

In general terms, leaf-wise algorithms are more efficient, they converge much faster than depth-wise. However, it may cause over-fitting when the data is small or there are too many leaves.

### Evaluation
Now let's evaluate the model in the test set.

In [24]:
with Timer() as t:
    y_prob_xgb = np.clip(xgb_clf_pipeline.predict(X_test), 0.0001, 0.9999)

In [25]:
results_dict['xgb']['test_time'] = t.interval

In [26]:
with Timer() as t:
    y_prob_lgbm = np.clip(lgbm_clf_pipeline.predict(X_test), 0.0001, 0.9999)

In [27]:
results_dict['lgbm']['test_time'] = t.interval


### Metrics
We are going to obtain some metrics to evaluate the performance of each of the models.

In [28]:
#https://github.com/miguelgfierro/codebase/blob/master/python/machine_learning/metrics.py
def classification_metrics_binary(y_true, y_pred):
    m_acc = accuracy_score(y_true, y_pred)
    m_f1 = f1_score(y_true, y_pred)
    m_precision = precision_score(y_true, y_pred)
    m_recall = recall_score(y_true, y_pred)
    report = {'Accuracy':m_acc, 'Precision':m_precision, 'Recall':m_recall, 'F1':m_f1}
    return report

In [29]:
#https://github.com/miguelgfierro/codebase/blob/master/python/machine_learning/metrics.py
def classification_metrics_binary_prob(y_true, y_prob):
    m_auc = roc_auc_score(y_true, y_prob)
    report = {'AUC':m_auc}
    return report

In [30]:
def binarize_prediction(y, threshold=0.5):
    y_pred = np.where(y > threshold, 1, 0)
    return y_pred

In [31]:
y_pred_xgb = binarize_prediction(y_prob_xgb)
y_pred_lgbm = binarize_prediction(y_prob_lgbm)


In [32]:
report_xgb = classification_metrics_binary(y_test, y_pred_xgb)
report2_xgb = classification_metrics_binary_prob(y_test, y_prob_xgb)
report_xgb.update(report2_xgb)
print(report_xgb)

{'F1': 0.6750787049642375, 'Recall': 0.62135450290731631, 'Precision': 0.7389725187643259, 'AUC': 0.78702209458708339, 'Accuracy': 0.71609982426199514}


In [33]:
results_dict['xgb']['performance'] = report_xgb


In [34]:
report_lgbm = classification_metrics_binary(y_test, y_pred_lgbm)
report2_lgbm = classification_metrics_binary_prob(y_test, y_prob_lgbm)
report_lgbm.update(report2_lgbm)
print(report_lgbm)

{'F1': 0.69369725877053401, 'Recall': 0.63108226721970184, 'Precision': 0.77010596311617108, 'AUC': 0.80873261360501847, 'Accuracy': 0.73547341413005862}


In [35]:
results_dict['lgbm']['performance'] = report_lgbm

In [36]:
# Results
print(json.dumps(results_dict, indent=4, sort_keys=True))

{
    "lgbm": {
        "performance": {
            "AUC": 0.8087326136050185,
            "Accuracy": 0.7354734141300586,
            "F1": 0.693697258770534,
            "Precision": 0.7701059631161711,
            "Recall": 0.6310822672197018
        },
        "test_time": 284.91796549700666,
        "train_time": 593.9747829429689
    },
    "xgb": {
        "performance": {
            "AUC": 0.7870220945870834,
            "Accuracy": 0.7160998242619951,
            "F1": 0.6750787049642375,
            "Precision": 0.7389725187643259,
            "Recall": 0.6213545029073163
        },
        "test_time": 9.840801779995672,
        "train_time": 5941.09022955902
    }
}


The experiment shows a fairly similar performance in both libraries, being LightGBM slightly better.

# 2) Concept drift
In this section we are trying to find concept drift in the dataset to check if retraining is valuable.

### Data management
We are going to pack the data yearly to try to find concept drift

In [37]:
def get_data_list_yearly(df):
    data_yearly = [df[df['Year'] == year] for year in range(1987, 2008)]  
    return data_yearly

In [38]:
%%time
data_yearly_list = get_data_list_yearly(df_plane_numeric)
for subset in data_yearly_list:
    print(subset.shape)

(1287333, 15)
(5126498, 15)
(290827, 15)
(5110527, 15)
(4995005, 15)
(5020651, 15)
(4993587, 15)
(5078411, 15)
(5219140, 15)
(5209326, 15)
(5301999, 15)
(5227051, 15)
(5360018, 15)
(5481303, 15)
(5723673, 15)
(5197860, 15)
(6375689, 15)
(6987729, 15)
(6992838, 15)
(7003802, 15)
(7275288, 15)
CPU times: user 6.59 s, sys: 4.37 s, total: 11 s
Wall time: 11.2 s


In [39]:
total_subsets = len(data_yearly_list)
print("Number of years: {}".format(total_subsets))
num_ini = 5

Number of years: 21


In [40]:
def generate_subset(data_yearly_list, num):
    subset = data_yearly_list[0]
    for i in range(1,num):
        subset = pd.concat([subset, data_yearly_list[i]])
    return subset

### Traininig
Let's see what happens when we train on a subset of data and then evaluate in the data of the following years.

In [41]:
%%time
subset_base = generate_subset(data_yearly_list, num_ini)
print(subset_base.shape)

(16810190, 15)
CPU times: user 700 ms, sys: 1.09 s, total: 1.79 s
Wall time: 1.79 s


In [42]:
X_train, y_train = generate_feables(subset_base)

In [43]:
clf = LGBMClassifier(num_leaves=255,
                    n_estimators=100,
                    min_child_weight=30,
                    learning_rate=0.1,
                    subsample=0.80,
                    colsample_bytree=0.80,
                    seed=42)

In [44]:
%%time
clf.fit(X_train, y_train)

CPU times: user 26min 20s, sys: 2min 15s, total: 28min 36s
Wall time: 1min 19s


LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.8, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.1, max_bin=255, max_depth=-1,
        max_drop=50, min_child_samples=10, min_child_weight=30,
        min_split_gain=0, n_estimators=100, nthread=-1, num_leaves=255,
        objective='binary', reg_alpha=0, reg_lambda=0, scale_pos_weight=1,
        seed=42, sigmoid=1.0, silent=True, skip_drop=0.5, subsample=0.8,
        subsample_for_bin=50000, subsample_freq=1, uniform_drop=False,
        xgboost_dart_mode=False)

In [45]:
def predict_accuracy_future_years(clf, data_yearly_list, num_ini):
    total_subsets = len(data_yearly_list)
    accuracy_dict = {}
    for y in range(num_ini, total_subsets):
        year = data_yearly_list[y]['Year'].iloc[0]
        print("Predicting year {}...".format(year))
        X_test, y_test = generate_feables(data_yearly_list[y])
        y_pred = clf.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        accuracy_dict[year] = acc
    return accuracy_dict

In [46]:
%%time
accuracy_dict = predict_accuracy_future_years(clf, data_yearly_list, num_ini)
print(accuracy_dict)

Predicting year 1992...
Predicting year 1993...
Predicting year 1994...
Predicting year 1995...
Predicting year 1996...
Predicting year 1997...
Predicting year 1998...
Predicting year 1999...
Predicting year 2000...
Predicting year 2001...
Predicting year 2002...
Predicting year 2003...
Predicting year 2004...
Predicting year 2005...
Predicting year 2006...
Predicting year 2007...
{1992: 0.75635888652686678, 1993: 0.75543231749041317, 1994: 0.74359046560036202, 1995: 0.7311463191253732, 1996: 0.72234911003841951, 1997: 0.71980285171687131, 1998: 0.70528162055430488, 1999: 0.70001910441345527, 2000: 0.68899894787790417, 2001: 0.67335170964518765, 2002: 0.67853347339097247, 2003: 0.68455330866985509, 2004: 0.68012640444413341, 2005: 0.67296453886104612, 2006: 0.66332086486739628, 2007: 0.65060764604782662}
CPU times: user 7h 37min 14s, sys: 27min 56s, total: 8h 5min 11s
Wall time: 21min 44s


From the results we can observe that the accuracy of the model gets worse as the years pass on.

### Retraining
Now let's see what happens when we retrain and evaluate in the data of the following years.

In [47]:
new_init = 15

In [48]:
%%time
subset_retrain = generate_subset(data_yearly_list, new_init)
print(subset_retrain.shape)

(69425349, 15)
CPU times: user 10.7 s, sys: 38.4 s, total: 49 s
Wall time: 49.5 s


In [49]:
X_train, y_train = generate_feables(subset_retrain)

In [50]:
clf_retrain = LGBMClassifier(num_leaves=255,
                    n_estimators=100,
                    min_child_weight=30,
                    learning_rate=0.1,
                    subsample=0.80,
                    colsample_bytree=0.80,
                    seed=42)

In [ ]:
%%time
clf_retrain.fit(X_train, y_train)

In [ ]:
%%time
accuracy_retrain = predict_accuracy_future_years(clf_retrain, data_yearly_list, new_init)
print(accuracy_retrain)

### Plot

In [ ]:
def plot_metrics(metric1, metric2, legend1=None, legend2=None, x_label=None, y_label=None):
    lists = sorted(metric1.items()) 
    x, y = zip(*lists) 
    fig, ax = plt.subplots()
    ax.plot(x, y, label=legend1)
    lists2 = sorted(metric2.items()) 
    x2, y2 = zip(*lists2) 
    ax.plot(x2, y2, label=legend2)
    legend = ax.legend(loc=0)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    plt.show()

In [ ]:
# Airline Retrain Results
plot_metrics(accuracy_dict, accuracy_retrain, legend1='Accuracy train', legend2='Accuracy retrain', x_label='year', 
             y_label='Accuracy')

As it can be seen, the performance is better after retraining. We have found concept drift in this dataset.